In [17]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np 

tokenizer = Tokenizer()

def dataset_preparation(data):

    # basic cleanup
    corpus = data.lower().split("\n")

    # tokenization	
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    # create input sequences using list of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # pad sequences 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    # create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)

    return predictors, label, max_sequence_len, total_words

def create_model(predictors, label, max_sequence_len, total_words):

    model = Sequential()
    model.add(Embedding(total_words, 15, input_length=max_sequence_len-1))
    model.add(LSTM(700, return_sequences = True))
    model.add(LSTM(200))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    model.fit(predictors, label, epochs=200, verbose=1, callbacks=[earlystop])
    model.summary()
    return model 

def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text



data = open('data.txt').read()

predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)
generate_text("Harry said", 10, max_sequence_len)

Epoch 1/200
146/146 [==============================] - 14s 98ms/step - loss: 4.5962 - acc: 0.0274
Epoch 2/200
146/146 [==============================] - 9s 59ms/step - loss: 4.5892 - acc: 0.0616
Epoch 3/200
146/146 [==============================] - 8s 54ms/step - loss: 4.4825 - acc: 0.0685
Epoch 4/200
146/146 [==============================] - 8s 53ms/step - loss: 4.4664 - acc: 0.0685
Epoch 5/200
146/146 [==============================] - 8s 54ms/step - loss: 4.3969 - acc: 0.0685
Epoch 6/200
146/146 [==============================] - 8s 56ms/step - loss: 4.3794 - acc: 0.0685
Epoch 7/200
146/146 [==============================] - 8s 55ms/step - loss: 4.3673 - acc: 0.0685
Epoch 8/200
146/146 [==============================] - 8s 58ms/step - loss: 4.3513 - acc: 0.0685
Epoch 9/200
146/146 [==============================] - 8s 52ms/step - loss: 4.3455 - acc: 0.0685
Epoch 10/200
146/146 [==============================] - 8s 52ms/step - loss: 4.3423 - acc: 0.0685
Epoch 11/200
146/146 [======

146/146 [==============================] - 7s 49ms/step - loss: 0.7532 - acc: 0.8904
Epoch 85/200
146/146 [==============================] - 7s 47ms/step - loss: 0.7233 - acc: 0.9041
Epoch 86/200
146/146 [==============================] - 7s 48ms/step - loss: 0.7028 - acc: 0.9110
Epoch 87/200
146/146 [==============================] - 7s 47ms/step - loss: 0.6914 - acc: 0.8767
Epoch 88/200
146/146 [==============================] - 7s 46ms/step - loss: 0.6737 - acc: 0.8904
Epoch 89/200
146/146 [==============================] - 7s 49ms/step - loss: 0.6552 - acc: 0.9041
Epoch 90/200
146/146 [==============================] - 7s 47ms/step - loss: 0.6404 - acc: 0.9110
Epoch 91/200
146/146 [==============================] - 7s 47ms/step - loss: 0.6491 - acc: 0.9041
Epoch 92/200
146/146 [==============================] - 7s 51ms/step - loss: 0.6234 - acc: 0.9110
Epoch 93/200
146/146 [==============================] - 7s 48ms/step - loss: 0.6150 - acc: 0.9178
Epoch 94/200
146/146 [===========

146/146 [==============================] - 7s 48ms/step - loss: 0.1291 - acc: 1.0000
Epoch 168/200
146/146 [==============================] - 7s 49ms/step - loss: 0.1260 - acc: 1.0000
Epoch 169/200
146/146 [==============================] - 7s 51ms/step - loss: 0.1253 - acc: 0.9932
Epoch 170/200
146/146 [==============================] - 7s 46ms/step - loss: 0.1225 - acc: 1.0000
Epoch 171/200
146/146 [==============================] - 8s 53ms/step - loss: 0.1170 - acc: 1.0000
Epoch 172/200
146/146 [==============================] - 7s 49ms/step - loss: 0.1189 - acc: 0.9932
Epoch 173/200
146/146 [==============================] - 7s 50ms/step - loss: 0.1152 - acc: 0.9932
Epoch 174/200
146/146 [==============================] - 8s 56ms/step - loss: 0.1128 - acc: 1.0000
Epoch 175/200
146/146 [==============================] - 6s 40ms/step - loss: 0.1075 - acc: 1.0000
Epoch 176/200
146/146 [==============================] - 6s 39ms/step - loss: 0.1040 - acc: 1.0000
Epoch 177/200
146/146 [=

'Harry said thing you shouldn’t harry said to ron and hermione i'